## MTCNN

In [3]:
from mtcnn import MTCNN
import cv2
import os

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] =str(0)

In [4]:
PATH_FOLDER = '/home/mhkim/temp'
walk_path = os.walk(PATH_FOLDER)
imgs = []
for a,b,c in walk_path:
    for _c in c:
        if '.png' in _c:
            imgs.append(os.path.join(a,_c)) 

In [13]:
sorted(imgs)
imgs = imgs[:len(imgs)//2]

### TEST

## MTCNN WITH FACE CROP

In [14]:
detector = MTCNN()

In [4]:
for item in imgs:
    img = cv2.cvtColor(cv2.imread(item), cv2.COLOR_BGR2RGB)
    
    info = detector.detect_faces(img)
    box = info[0]['box']
    crop_img = img[box[1]:box[1]+box[3],box[0]:box[0]+box[2] ]#슬라이싱
    crop_img = cv2.cvtColor(crop_img,cv2.COLOR_BGR2RGB)

    SAVE_PATH = item.replace('temp','temp_crop')
    os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
    cv2.imwrite(SAVE_PATH, crop_img)


NameError: name 'imgs' is not defined

In [ ]:
import matplotlib.pyplot as plt
# plt.imshow(img)
plt.imshow(crop_img)

NameError: name 'crop_img' is not defined

In [27]:
clist=[]
PATH_FOLDER = ''
for a,b,c in os.walk(PATH_FOLDER):
    for _c in c:
        clist.append(os.path.join(a,_c))

## Faster RCNN

In [1]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 3090


In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [5]:

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [15]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    print(preds)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

In [7]:
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision import transforms

dir_testset = '/home/mhkim/temp/test'
val_aug = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
])
    
loader_test = DataLoader(datasets.ImageFolder(dir_testset, val_aug),
                                        batch_size=1,
                                        shuffle=False,
                                        num_workers=4,
                                        pin_memory=True
                                        )

In [17]:
with torch.no_grad(): 
    # 테스트셋 배치사이즈= 2
    for imgs, annotations in loader_test:
        print(annotations)
        imgs = list(img.to(device) for img in imgs)
#         print(imgs)
        pred = make_prediction(model, imgs, 0.5)
        print(pred)
        break

tensor([0])
[{'boxes': tensor([[ 51.8174,  69.6426,  82.7765,  82.2821],
        [ 22.5271,   2.2172,  57.9255,  18.2714],
        [ 43.7006,  69.0759,  74.1748,  86.1525],
        [ 23.8653,  50.8138,  95.6948,  79.6754],
        [ 47.0850,  56.5907,  78.5868,  71.8142],
        [ 57.5480,  12.4111, 126.3008,  43.5582],
        [ 54.1073,  52.4287,  74.0716,  70.4610],
        [ 35.0091,  43.9845, 116.9516,  72.0033],
        [ 33.4368,  51.9333,  86.8937, 110.6711],
        [  3.9392,  76.8534,  54.3595,  98.1538],
        [ 24.0670,  59.0396,  88.8453,  89.4175],
        [ 68.3897,  39.6818, 127.8681,  66.3557],
        [ 79.0690, 114.9231,  83.5180, 119.5103],
        [ 81.5017, 115.3152,  86.0535, 120.3772],
        [ 68.9318,  49.5238, 127.7381,  73.8343],
        [ 66.3139,  40.0537, 127.2350,  99.3583],
        [  7.2620,  79.7959,  67.1675, 106.2580],
        [ 78.4287, 113.3761,  86.0267, 122.1207],
        [ 62.1472,  60.1612, 126.4062,  85.0884],
        [ 53.5306,   9.1006